This is a wish-list of models to still be implemented computationally (from Mellow & Beranek 2012):

> circular piston in a closed circular baffle
> circular piston in a rigid sphere
> 

### Circular piston in a rigid sphere

eqn. 12.111: $D(\theta)=-\frac{4}{k^{2}R^{2}sin^{2}\alpha} \sum_{n=0}^{N} A_{n}j^{n}P_{n}cos(\theta) $

* Here the unknown is $A_{n}$
    * There is an $A_{n}$ defined in eqn. 12.102 -- it is the result of a matrix inversion $a=M^{-1}.b$

From the explanations of 12.7-12.8 we know tht $P_{n}$ is the legendre function, $j_{n}$ is the Bessel function and $h_{n}^{(2)}$ is the spherical Hankel function (eqn.2.18)

In [1]:
import numpy as np
from scipy.special import jv as bessel_firstkind
from sympy import besselj, symbols, hankel2, legendre, sin, cos, tan, summation, I, oo, diff, pi
from sympy import Abs, lambdify, integrate

In [10]:
alpha, m, r1, R, theta = symbols('alpha m theta r1 R')
r1 = (R*cos(alpha))/cos(theta)

In [11]:
# some constants here : 
#R = 0.1
#alpha = np.radians(20)



In [12]:
# eqn 12.108

def calculate_L_of_m(m, theta, r1, R):
    eqn12_108_expr = legendre(m, cos(theta))*(r1**2/R**2)*tan(theta)
    L = integrate(eqn12_108_expr, (theta,0,alpha))
    return L
#Lm_calc = lambdify([m, theta, r1, R], calculate_L_of_m)

In [19]:
etr = integrate(legendre(m, cos(theta))*(r1**2/R**2)*tan(theta), (theta,0,alpha)).sub({m:0,R:0.1,alpha:np.radians(10)})


KeyboardInterrupt: 